In [1]:
#pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 11.2 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install numpy


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Question_1: Image Annotation
parts 1.1 & 1.2

In [2]:
import cv2
import json
import urllib.request
import numpy as np

image_url = "http://images.cocodataset.org/val2017/000000184791.jpg"

def draw_rectangles(event, x, y, flags, param):
    global refPt, cropping, rectangles, img_copy

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        rectangles.append(refPt)
        color = (0, 255, 0)  # Green color
        cv2.rectangle(img_copy, refPt[0], refPt[1], color, 2)
        cv2.imshow("Select Objects", img_copy)


# after giving the URL for the image this is the code for Downland the image: 
req = urllib.request.urlopen(image_url)
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
image = cv2.imdecode(arr, -1)

# Make a copy of the image to draw rectangles on
img_copy = image.copy()

# List to hold the rectangles' coordinates
rectangles = []

# Create a window and set the callback function
cv2.namedWindow("Select Objects")
cv2.setMouseCallback("Select Objects", draw_rectangles)

# Keep looping until 'q' is pressed
while True:
    cv2.imshow("Select Objects", img_copy)
    key = cv2.waitKey(1) & 0xFF

    # Press 'r' to reset the rectangles drawn
    if key == ord("r"):
        img_copy = image.copy()
        rectangles = []

    # Press 'q' to quit the program and display the final image with rectangles
    elif key == ord("q"):
        break

# Construct the JSON object with annotations
annotations = []

for index, rect in enumerate(rectangles):
    x_min = min(rect[0][0], rect[1][0])
    y_min = min(rect[0][1], rect[1][1])
    x_max = max(rect[0][0], rect[1][0])
    y_max = max(rect[0][1], rect[1][1])

    annotation = {
        "label": f"object{index+1}",
        "coordinates": {
            "x_min": x_min,
            "y_min": y_min,
            "x_max": x_max,
            "y_max": y_max
        }
    }
    annotations.append(annotation)

# Save annotations to a JSON file
json_data = {
    "image_name": "image1.jpg",  # Replace with the actual image name
    "annotations": annotations
}

with open('bounding_boxes.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print("Bounding box coordinates saved in 'bounding_boxes.json' file.")

# Display the final image with rectangles
for rect in rectangles:
    color = (0, 255, 0)  # Green color
    cv2.rectangle(image, rect[0], rect[1], color, 2)

cv2.imshow("Final Image with Rectangles", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Bounding box coordinates saved in 'bounding_boxes.json' file.


### Question_2: Scripting


#### question 2.1:

In [13]:
import cv2
import json
import os
import csv

def normalize_coordinates(image_width, image_height, x_min, y_min, x_max, y_max):
    # Normalize coordinates by dividing by the image dimensions
    x_min_norm = x_min / image_width
    y_min_norm = y_min / image_height
    x_max_norm = x_max / image_width
    y_max_norm = y_max / image_height
    return x_min_norm, y_min_norm, x_max_norm, y_max_norm

def convert_to_csv(image_path, annotation_json):
    if not os.path.exists(image_path):
        print("Image file not found.")
        return

    if not os.path.exists(annotation_json):
        print("Annotation file not found.")
        return

    image = cv2.imread(image_path)
    if image is None:
        print("Could not read the image. Please check the file path.")
        return

    image_height, image_width, _ = image.shape

    with open(annotation_json, 'r') as json_file:
        data = json.load(json_file)

    csv_file = 'normalized_bounding_boxes.csv'

    with open(csv_file, mode='w', newline='') as csv_file:
        fieldnames = ['label', 'x_min', 'y_min', 'x_max', 'y_max']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for annotation in data['annotations']:
            label = annotation['label']
            x_min = annotation['coordinates']['x_min']
            y_min = annotation['coordinates']['y_min']
            x_max = annotation['coordinates']['x_max']
            y_max = annotation['coordinates']['y_max']

            # Normalize coordinates
            x_min_norm, y_min_norm, x_max_norm, y_max_norm = normalize_coordinates(
                image_width, image_height, x_min, y_min, x_max, y_max)

            writer.writerow({
                'label': label,
                'x_min': x_min_norm,
                'y_min': y_min_norm,
                'x_max': x_max_norm,
                'y_max': y_max_norm
            })

    print(f"Normalized bounding boxes saved in '{csv_file}'")

#links to the Path's:
image_file = '/Users/razherman/GM Task/000000184791.jpg'  
json_annotation = '/Users/razherman/GM Task/bounding_boxes.json'  

convert_to_csv(image_file, json_annotation)



Normalized bounding boxes saved in '<_io.TextIOWrapper name='normalized_bounding_boxes.csv' mode='w' encoding='UTF-8'>'


#### question 2.2:

In [14]:
import cv2
import json
import os
import csv

def normalize_coordinates(image_width, image_height, x_min, y_min, x_max, y_max):
    # Normalize coordinates by dividing by the image dimensions
    x_min_norm = x_min / image_width
    y_min_norm = y_min / image_height
    x_max_norm = x_max / image_width
    y_max_norm = y_max / image_height
    return x_min_norm, y_min_norm, x_max_norm, y_max_norm

def convert_to_csv_and_json(image_path, annotation_json):
    if not os.path.exists(image_path):
        print("Image file not found.")
        return

    if not os.path.exists(annotation_json):
        print("Annotation file not found.")
        return

    image = cv2.imread(image_path)
    if image is None:
        print("Could not read the image. Please check the file path.")
        return

    image_height, image_width, _ = image.shape

    with open(annotation_json, 'r') as json_file:
        data = json.load(json_file)

    csv_file = 'normalized_bounding_boxes.csv'
    json_output = 'converted_coordinates.json'

    with open(csv_file, mode='w', newline='') as csv_file:
        fieldnames = ['label', 'x_min', 'y_min', 'x_max', 'y_max']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        with open(json_output, 'w') as json_out:
            converted_annotations = {
                "image_name": data["image_name"],
                "annotations": []
            }

            for annotation in data['annotations']:
                label = annotation['label']
                x_min = annotation['coordinates']['x_min']
                y_min = annotation['coordinates']['y_min']
                x_max = annotation['coordinates']['x_max']
                y_max = annotation['coordinates']['y_max']

                # Normalize coordinates
                x_min_norm, y_min_norm, x_max_norm, y_max_norm = normalize_coordinates(
                    image_width, image_height, x_min, y_min, x_max, y_max)

                writer.writerow({
                    'label': label,
                    'x_min': x_min_norm,
                    'y_min': y_min_norm,
                    'x_max': x_max_norm,
                    'y_max': y_max_norm
                })

                # Convert to x_min, y_min, width, height format for JSON output
                width = x_max - x_min
                height = y_max - y_min

                converted_annotations["annotations"].append({
                    "label": label,
                    "coordinates": {
                        "x_min": x_min,
                        "y_min": y_min,
                        "width": width,
                        "height": height
                    }
                })

            json.dump(converted_annotations, json_out, indent=4)

    print(f"Normalized bounding boxes saved in '{csv_file}'")
    print(f"Converted coordinates saved in '{json_output}'")

#links to the Path's:
image_file = '/Users/razherman/GM Task/000000184791.jpg'  
json_annotation = '/Users/razherman/GM Task/bounding_boxes.json' 

convert_to_csv_and_json(image_file, json_annotation)


Normalized bounding boxes saved in '<_io.TextIOWrapper name='normalized_bounding_boxes.csv' mode='w' encoding='UTF-8'>'
Converted coordinates saved in 'converted_coordinates.json'


#### question 2.3:

In [16]:
import cv2
import json
import os
import csv

def normalize_coordinates(image_width, image_height, x_min, y_min, x_max, y_max):
    x_min_norm = x_min / image_width
    y_min_norm = y_min / image_height
    x_max_norm = x_max / image_width
    y_max_norm = y_max / image_height
    return x_min_norm, y_min_norm, x_max_norm, y_max_norm

def convert_to_csv_and_json(image_path, annotation_json):
    if not os.path.exists(image_path):
        print("Image file not found.")
        return

    if not os.path.exists(annotation_json):
        print("Annotation file not found.")
        return

    image = cv2.imread(image_path)
    if image is None:
        print("Could not read the image. Please check the file path.")
        return

    image_copy = image.copy()  # Make a copy to draw bounding boxes

    image_height, image_width, _ = image.shape

    with open(annotation_json, 'r') as json_file:
        data = json.load(json_file)

    csv_file = 'normalized_bounding_boxes.csv'
    json_output = 'converted_coordinates.json'

    with open(csv_file, mode='w', newline='') as csv_file:
        fieldnames = ['label', 'x_min', 'y_min', 'x_max', 'y_max']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        with open(json_output, 'w') as json_out:
            converted_annotations = {
                "image_name": data["image_name"],
                "annotations": []
            }

            for annotation in data['annotations']:
                label = annotation['label']
                x_min = annotation['coordinates']['x_min']
                y_min = annotation['coordinates']['y_min']
                x_max = annotation['coordinates']['x_max']
                y_max = annotation['coordinates']['y_max']

                # Normalize coordinates
                x_min_norm, y_min_norm, x_max_norm, y_max_norm = normalize_coordinates(
                    image_width, image_height, x_min, y_min, x_max, y_max)

                writer.writerow({
                    'label': label,
                    'x_min': x_min_norm,
                    'y_min': y_min_norm,
                    'x_max': x_max_norm,
                    'y_max': y_max_norm
                })

                # Convert to x_min, y_min, width, height format for JSON output
                width = x_max - x_min
                height = y_max - y_min

                converted_annotations["annotations"].append({
                    "label": label,
                    "coordinates": {
                        "x_min": x_min,
                        "y_min": y_min,
                        "width": width,
                        "height": height
                    }
                })

                # Draw bounding boxes on the image
                color = (0, 255, 0)
                cv2.rectangle(image_copy, (x_min, y_min), (x_max, y_max), color, 2)

            json.dump(converted_annotations, json_out, indent=4)

    # Display the images with bounding boxes
    cv2.imshow("Normalized Bounding Boxes", image_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# links to the Path's:
image_file = '/Users/razherman/GM Task/000000184791.jpg' 
json_annotation = '/Users/razherman/GM Task/bounding_boxes.json'  

convert_to_csv_and_json(image_file, json_annotation)
